In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
import pandas as pd
import numpy as np

In [2]:
path = 'D:\Study\Iris dataset\Iris.csv'
df = pd.read_csv(path)
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

class IrisDS(Dataset):
    def __init__(self, data, labels):
        super().__init__()
        self.data = data
        self.labels = labels
    
    def __getitem__(self, ix):
        data = df[['SepalLengthCm', 'SepalWidthCm','PetalLengthCm','PetalWidthCm']].values[ix]
        features = torch.tensor(data)
        label = df['Species'][ix]
        label = self.labels[label]
        return features, label
    
    def __len__(self):
        return len(self.data)
    
class IrisDM(pl.LightningDataModule):
    def __init__(self, df, labels):
        super().__init__()
        self.trn, self.val = train_test_split(df)
        self.trn_dataset = IrisDS(self.trn, labels)
        self.val_dataset = IrisDS(self.val, labels)
        
    def train_dataloader(self):
        return DataLoader(self.trn_dataset, batch_size=32, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=32)
        
        
labels = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
dm = IrisDM(df, labels)
a = iter(dm.train_dataloader())
b = next(a)
b

[tensor([[5.1000, 3.4000, 1.5000, 0.2000],
         [4.8000, 3.0000, 1.4000, 0.1000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.5000, 2.6000, 4.4000, 1.2000],
         [5.0000, 3.5000, 1.3000, 0.3000],
         [6.8000, 2.8000, 4.8000, 1.4000],
         [6.3000, 2.3000, 4.4000, 1.3000],
         [5.1000, 3.8000, 1.6000, 0.2000],
         [5.6000, 2.5000, 3.9000, 1.1000],
         [5.4000, 3.9000, 1.3000, 0.4000],
         [5.9000, 3.2000, 4.8000, 1.8000],
         [5.2000, 4.1000, 1.5000, 0.1000],
         [4.9000, 2.4000, 3.3000, 1.0000],
         [5.0000, 3.2000, 1.2000, 0.2000],
         [6.0000, 2.9000, 4.5000, 1.5000],
         [5.1000, 3.5000, 1.4000, 0.3000],
         [4.8000, 3.0000, 1.4000, 0.3000],
         [5.0000, 3.0000, 1.6000, 0.2000],
         [5.4000, 3.7000, 1.5000, 0.2000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [6.7000, 2.5000, 5.8000, 1.8000],
         [5

In [4]:
class Net(pl.LightningModule):
    def linear_layer(self,ni,no):
        return nn.Sequential(
            nn.Linear(ni,no),
            nn.ReLU(inplace=True)
        )
    
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            self.linear_layer(4,16),
            self.linear_layer(16,32)
        )
        self.lin = nn.Linear(32,3)
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, x):
        x = self.fc(x)
        x = self.lin(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x.float())
        loss = self.loss_fn(y_hat, y)
        matches = [torch.argmax(i) == j for i,j in zip(y_hat,y)]
        acc = sum(matches)/len(matches)
        self.log('acc', acc, on_step=True, on_epoch=True,
                prog_bar=True, logger=True)
        return {'loss':loss, 'acc': acc}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x.float())
        loss = self.loss_fn(y_hat, y)
        matches = [torch.argmax(i) == j for i,j in zip(y_hat,y)]
        val_acc = sum(matches)/len(matches)
        self.log('val_acc', val_acc, on_step=True, on_epoch=True,
                prog_bar=True, logger=True)
        return {'loss':loss, 'val_acc': val_acc}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)
    
    def predict(self, test):
        if len(test) == 4:
            pred = self(test.float())
            return torch.argmax(pred).item()
        else:
            pred = self(test.float())
            return torch.argmax(pred, dim=1)
    
    def evaluate(self, testx, labels):
        preds = self.predict(testx)
        if isinstance(preds, int):
            return preds==labels
        else:
            matches = (preds==labels)
            acc = sum(matches)/len(matches)
            return acc.item()
        
    def get_progress_bar_dict(self):
        tqdm_dict = super().get_progress_bar_dict()
        if 'v_num' in tqdm_dict:
            del tqdm_dict['v_num']
        return tqdm_dict

In [5]:
if __name__ == '__main__':
    net = Net()
    trainer = pl.Trainer(max_epochs=100, gpus=[0])
    trainer.fit(net, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | fc      | Sequential       | 624   
1 | lin     | Linear           | 99    
2 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
723       Trainable params
0         Non-trainable params
723       Total params


In [6]:
testset = dm.val_dataset
rnd = np.random.randint(0, len(testset))
testx = testset[rnd][0]
testy = testset[rnd][1]

batchset = iter(dm.val_dataloader())
batchx, batchy = next(batchset)

pred = net.predict(batchx)
acc = net.evaluate(batchx, batchy)
acc

1.0